In [ ]:
import redis
import io
import os
import json
from redis.commands.search.query import Query
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.field import TextField, TagField, NumericField, VectorField
import numpy as np
from numpy.linalg import norm
from sentence_transformers import SentenceTransformer

In [ ]:
# Get a Redis connection
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

# Define the model we want to use 
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
# Reading the books and storing as JSON documents together with the corresponding vector embedding
# The embedding is calculated using the description.
for filename in os.listdir("../../data/books/"):
    f = os.path.join("../../data/books/", filename)

    if os.path.isfile(f):
        book_file = io.open(f, encoding="utf-8")
        book_json = json.load(book_file)
        book_file.close()
        #book_json['embedding'] = model.encode(book_json['description']).astype(np.float32).tolist()
        r.json().set(f"book:{book_json['id']}", "$", book_json)
        r.json().set(f"book:{book_json['id']}", "$.embedding", model.encode(book_json['description']).astype(np.float32).tolist())

In [ ]:
# Here we create an index if it does not exists
indexes = r.execute_command("FT._LIST")
if "books_idx" not in indexes: #.encode("utf-8")
    index_def = IndexDefinition(prefix=["book:"], index_type=IndexType.JSON)
    schema = (TagField("$.title", as_name="title"),
              TagField("$.status", as_name="status"),
              TagField("$.author", as_name="author"),
              NumericField("$.year_published", as_name="year_published"),
              VectorField("$.embedding", "HNSW", {"TYPE": "FLOAT32", "DIM": 384, "DISTANCE_METRIC": "COSINE", "INITIAL_CAP": 1500}, as_name="embedding"))
    r.ft('books_idx').create_index(schema, definition=index_def)

In [ ]:
def get_recommendation2(key):
    q = Query("@embedding:[VECTOR_RANGE $radius $vec]=>{$YIELD_DISTANCE_AS: score}") \
        .return_fields("title") \
        .paging(0, 3) \
        .dialect(2)

    # Find all vectors within VSS_MINIMUM_SCORE of the query vector
    query_params = {
        "radius": 5,
        "vec": model.encode("recommend a science fiction book").astype(np.float32).tobytes()
    }

    res = r.ft("books_idx").search(q, query_params)
    return res

In [ ]:
# Get the recommendation from a book 
def get_recommendation(key):
    embedding = r.json().get(key)
    embedding_as_blob = np.array(embedding['embedding'], dtype=np.float32).tobytes()
    q = Query("*=>[KNN 5 @embedding $vec AS score]").return_field("$.title").sort_by("score", asc=True).dialect(2).paging(1, 5)
    res = r.ft("books_idx").search(q, query_params={"vec": embedding_as_blob})
    return res

In [ ]:
print(get_recommendation('book:26415'))
print(get_recommendation('book:9'))